In [68]:
import math
import numpy as np

class QFromat:
    
    def __init__(self, m: int, n: int, U: bool=False):
        '''
        :param m: Integer bits 
        :param n: Fractional bits
        :param U: Signed?
        '''
        self.m = m
        self.n = n
        self.size = m + n 
        self.signed = U
        
        if(self.signed):
            self.minimum = 0
            self.maximum = pow(2, self.m) - pow(2, -self.n)
            self.resolution = pow(2, -self.n)
            self.wordSize = pow(2, self.size)
            self.decRange = [self.minimum, self.maximum]
            self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]
        else:
            self.minimum = -pow(2, self.m - 1)
            self.maximum = pow(2, self.m - 1) - pow(2, -self.n)
            self.resolution = pow(2, -self.n)
            self.wordSize = pow(2, self.size)
            self.decRange = [self.minimum, self.maximum]
            self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]    
        
        self.headroom = (self.maximum - self.minimum + self.resolution)
            
#     def Q(self, m, n):
#         self.minimum = -pow(2, m - 1)
#         self.maximum = pow(2, m - 1) - pow(2, -n)
#         self.resolution = pow(2, -n)
#         self.wordSize = pow(2, m + n)
#         self.decRange = [self.minimum, self.maximum]
#         self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]    
    
#     def UQ(self, m, n):
#         self.minimum = 0
#         self.maximum = pow(2, m) - pow(2, -n)
#         self.resolution = pow(2, -n)
#         self.wordSize = pow(2, m + n)
#         self.decRange = [self.minimum, self.maximum]
#         self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]
    
    def RtoF(self, input_data: hex) -> float:
        if(self.wordSize > input_data > self.maximum / self.resolution):
            return input_data * self.resolution - self.wordSize * self.resolution
        elif(self.maximum / self.resolution >= input_data >= 0):
            return input_data * self.resolution
        else:
            # overflow condition
            pass
    
    def FtoR(self, input_data: float) -> hex:
        if(self.maximum >= input_data >= 0):        
            input_data = (input_data / self.resolution)
            return hex(round(input_data))
        elif(self.minimum <= input_data < 0):
            input_data = (input_data / self.resolution)
            return hex(round(input_data) + self.wordSize)
        else:
            # overflow condition
            pass
        
    def Error(self, input_data: float) -> None:
        # Representation Error
        integer = self.FtoR(input_data)
        representation = self.RtoF(int(integer, 16))
        error = abs(input_data - representation)
        print("Input Value         : ", input_data)
        print("Integer Value       : ", integer)
        print("Fixed-point Value   : ", representation)
        print("Representation Error: ", error)
        print("Headroom            : ", self.headroom)
        
        if(abs(error) == 0):
            print("dBFS                : ", 0)
        else:
            print("dBFS                : ", round(20 * math.log(error/self.headroom, 10), 2))


In [69]:
obj = QFromat(2, 30, 0)
obj.Error(0.12)

Input Value         :  0.12
Integer Value       :  0x7ae147b
Fixed-point Value   :  0.12000000011175871
Representation Error:  1.1175871339474952e-10
Headroom            :  4.0
dBFS                :  -211.08


In [30]:
# print(obj.FtoR(-2.5))
# print(obj.FtoR(-2))
# print(obj.FtoR(-1.5))
# print(obj.FtoR(-1))
# print(obj.FtoR(-0.5))
# print(obj.FtoR(0))
# print(obj.FtoR(0.5))
# print(obj.FtoR(1))
# print(obj.FtoR(1.5))
# print(obj.FtoR(2.0))

In [4]:
# for i in range(64):
#     print(i, hex(i), obj.RtoF(i)) 